In [1]:
# Import the data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder


df = pd.read_csv('dataResource/valentine_gift_large_dataset.csv')
df.head()

,Recipient_Gender,Recipient_Age,Relationship_Status,Budget,Occasion,Preference,Relationship_Length,Personal_Interest,Special_Note,Gift_Popularity,Gift_Purpose,Past_Gift_Reaction,Delivery_Method,Gift_Wrapping_Preference,Best_Gift
0,Female,29,Single,High ($100-$500),Anniversary Valentine,Sentimental,<6 months,Fitness,Has allergies,Very Popular,Celebration,Disliked,Online Delivery,Fancy Wrapping,Perfume
1,Other,34,Married,Medium ($20-$100),First Valentine,Sentimental,3+ years,Music,Handmade gifts appreciated,Moderately Popular,Celebration,Highly Appreciated,Planned Surprise,No Wrapping,Fashion Accessories
2,Other,56,Single,Low (<$20),First Valentine,Practical,3+ years,Fashion,Tech-savvy,Moderately Popular,Expression of Love,Never Given Gift Before,Online Delivery,Eco-Friendly,Flowers
3,Other,50,Complicated,Luxury (>$500),First Valentine,Fun,6 months - 1 year,Cooking,Prefers practical gifts,Very Popular,Expression of Love,Disliked,Online Delivery,Fancy Wrapping,Fashion Accessories
4,Male,53,Married,Medium ($20-$100),Long-Distance Valentine,Surprise,<6 months,Music,Has allergies,Moderately Popular,Apology,Highly Appreciated,Planned Surprise,Eco-Friendly,Subscription Service


In [ ]:
# merge two samilar value to one vlaue
def mergeData(column_name, value_one, value_two):
    df[column_name].replace({value_one: value_two}, inplace=True)


mergeData("Recipient_Gender", 'Non-Binary', 'Other')
mergeData("Preference", "Sentimental", "Romantic")
mergeData("Personal_Interest", "Sports", "Fitness")
mergeData("Personal_Interest", "Gaming", "Technology")
mergeData("Best_Gift", "Handmade Gift", "Personalized Gift")
mergeData("Best_Gift", "Chocolates", "Chocolates & Flowers")
mergeData("Best_Gift", "Flowers", "Chocolates & Flowers")
mergeData("Best_Gift", "Jewelry", "Fashion Accessories")

df


,Recipient_Gender,Recipient_Age,Relationship_Status,Budget,Occasion,Preference,Relationship_Length,Personal_Interest,Special_Note,Gift_Popularity,Gift_Purpose,Past_Gift_Reaction,Delivery_Method,Gift_Wrapping_Preference,Best_Gift
0,Female,29,Single,High ($100-$500),Anniversary Valentine,Romantic,<6 months,Fitness,Has allergies,Very Popular,Celebration,Disliked,Online Delivery,Fancy Wrapping,Perfume
1,Other,34,Married,Medium ($20-$100),First Valentine,Romantic,3+ years,Music,Handmade gifts appreciated,Moderately Popular,Celebration,Highly Appreciated,Planned Surprise,No Wrapping,Fashion Accessories
2,Other,56,Single,Low (<$20),First Valentine,Practical,3+ years,Fashion,Tech-savvy,Moderately Popular,Expression of Love,Never Given Gift Before,Online Delivery,Eco-Friendly,Chocolates & Flowers
3,Other,50,Complicated,Luxury (>$500),First Valentine,Fun,6 months - 1 year,Cooking,Prefers practical gifts,Very Popular,Expression of Love,Disliked,Online Delivery,Fancy Wrapping,Fashion Accessories
4,Male,53,Married,Medium ($20-$100),Long-Distance Valentine,Surprise,<6 months,Music,Has allergies,Moderately Popular,Apology,Highly Appreciated,Planned Surprise,Eco-Friendly,Subscription Service
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,Other,43,Married,Luxury (>$500),Anniversary Valentine,Luxury,<6 months,Music,Enjoys luxury items,Moderately Popular,Celebration,Never Given Gift Before,Online Delivery,Minimalist,"Experience (Trip, Dinner, etc.)"
1996,Female,26,Engaged,Medium ($20-$100),First Valentine,Surprise,6 months - 1 year,Fashion,Handmade gifts appreciated,Moderately Popular,Expression of Love,Highly Appreciated,Courier,Minimalist,Subscription Service
1997,Other,50,Engaged,Luxury (>$500),First Valentine,Romantic,<6 months,Music,Loves surprises,Very Popular,Spontaneous Surprise,Never Given Gift Before,Planned Surprise,Minimalist,Subscription Service
1998,Other,30,Married,High ($100-$500),Casual Valentine,Romantic,6 months - 1 year,Technology,Enjoys luxury items,Rare Choice,Celebration,Disliked,Online Delivery,No Wrapping,Fashion Accessories


In [ ]:
# encode column's value


In [2]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split_marketing(df)
X_train.describe()

,age,balance,day,duration,campaign,pdays,previous
count,25431.000000,25431.000000,25431.000000,25431.000000,25431.000000,4655.000000,25431.000000
mean,40.913806,1353.540128,15.836381,257.774684,2.780937,226.843824,0.581534
std,10.617230,3018.184488,8.345164,258.556998,3.115439,117.716936,1.928374
min,18.000000,-8019.000000,1.000000,0.000000,1.000000,1.000000,0.000000
25%,33.000000,71.000000,8.000000,103.000000,1.000000,133.000000,0.000000
50%,39.000000,448.000000,16.000000,180.000000,2.000000,196.000000,0.000000
75%,48.000000,1403.000000,21.000000,318.000000,3.000000,330.500000,0.000000
max,95.000000,98417.000000,31.000000,3785.000000,63.000000,871.000000,51.000000


# Missing Values

In [3]:
# Fill the missing values using the imported function
X_train_filled = fill_missing(X_train)
X_test_filled = fill_missing(X_test)
X_train_filled.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
2544,34,blue-collar,married,secondary,no,328,yes,yes,cellular,21,nov,18,7,-1.0,0,nonexistent
3891,30,technician,married,secondary,no,484,yes,no,unknown,6,may,703,1,-1.0,0,nonexistent
19464,42,entrepreneur,divorced,secondary,no,31,no,no,unknown,18,jun,120,1,-1.0,0,nonexistent
31170,27,management,single,tertiary,no,3,yes,yes,cellular,22,jul,506,1,-1.0,0,nonexistent
22216,30,management,single,secondary,no,19,no,no,cellular,26,aug,191,2,-1.0,0,nonexistent


In [4]:
# Create the encoders for categorical variables (use X_train_filled)
encoders = build_encoders(X_train_filled)
encoders

[{'column': 'job',
  'multi_col_output': True,
  'encoder': OneHotEncoder(handle_unknown='infrequent_if_exist', max_categories=5,
                sparse_output=False)},
 {'column': 'marital',
  'multi_col_output': True,
  'encoder': OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False)},
 {'column': 'education',
  'multi_col_output': False,
  'encoder': OrdinalEncoder(categories=[['primary', 'secondary', 'tertiary']],
                 handle_unknown='use_encoded_value', unknown_value=-1)},
 {'column': 'default',
  'multi_col_output': False,
  'encoder': OrdinalEncoder(categories=[['no', 'yes']], handle_unknown='use_encoded_value',
                 unknown_value=-1)},
 {'column': 'housing',
  'multi_col_output': False,
  'encoder': OrdinalEncoder(categories=[['no', 'yes']], handle_unknown='use_encoded_value',
                 unknown_value=-1)},
 {'column': 'loan',
  'multi_col_output': False,
  'encoder': OrdinalEncoder(categories=[['no', 'yes']], handle_unknown='us

In [5]:
# Encode X_train_filled and X_test_filled
X_train_encoded = encode_categorical(X_train_filled, encoders)
X_test_encoded = encode_categorical(X_test_filled, encoders)

X_train_encoded.head()

,age,balance,day,duration,campaign,pdays,previous,x0_admin.,x0_blue-collar,x0_management,...,x0_telephone,x0_unknown,x0_nonexistent,x0_other,x0_success,education,default,housing,loan,month
0,34,328,21,18,7,-1.0,0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,10.0
1,30,484,6,703,1,-1.0,0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,4.0
2,42,31,18,120,1,-1.0,0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,5.0
3,27,3,22,506,1,-1.0,0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,1.0,6.0
4,30,19,26,191,2,-1.0,0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,7.0


In [6]:
# Encode y_train and y_test
y_encoder = build_target_encoder(y_train)
y_train_encoded = encode_target(y_train, y_encoder)
y_test_encoded = encode_target(y_test, y_encoder)

In [7]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, max_depth=7, random_state=13)
model.fit(X_train_encoded, y_train_encoded)
y_pred = model.predict(X_test_encoded)
print(balanced_accuracy_score(y_test_encoded, y_pred))

0.5753911268989494


In [8]:
# Import new data and test with the model

new_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m14/lesson_3/datasets/bank_marketing_new_data.csv')

In [9]:
X = new_df.drop(columns='y')
y = new_df['y'].values.reshape(-1, 1)

In [10]:
X_filled = fill_missing(X)
X_encoded = encode_categorical(X_filled, encoders)
y_encoded = encode_target(y, y_encoder)

In [11]:
y_pred = model.predict(X_encoded)
print(balanced_accuracy_score(y_encoded, y_pred))

0.5769138944243607
